In [1]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz

In [2]:
roads = pd.read_csv('Railroads.csv')
dests = pd.read_csv('destinations.csv')

In [3]:
roads.head()

,from,to,length,color
0,Seattle,Calgary,4,gray
1,Seattle,Helena,6,yellow
2,Portland,Salt Lake City,6,blue
3,Montreal,Sault St. Marie,5,black
4,Pittsburgh,Chicago,3,orange


In [4]:
dests.head()

,from,to,points
0,Boston,Miami,12
1,Calgary,Phoenix,13
2,Calgary,Salt Lake City,7
3,Chicago,New Orleans,7
4,Chicago,Santa Fe,9


In [5]:
# merege df columns from and to
def merge_unique_from_to(df):
    return np.concatenate((df['from'].values,df['to'].values))

# find distinct  in each of them, make sure there are no typos
def show_most_simelar_strings(container):
    droped_duplicates = tuple(set(container))
    results=[]
    for index,string_a in enumerate(droped_duplicates):
        for string_b in droped_duplicates[index+1:]:
            compere = (string_a,string_b,fuzz.ratio(string_a,string_b))
            results.append(compere)
    return sorted(results,key= lambda x: x[2],reverse=True)

In [6]:
for dataset in(roads,dests):
    all_city_names = merge_unique_from_to(dataset)
    most_simelar = show_most_simelar_strings(all_city_names)
    print(most_simelar[:10])

[('Boston', 'Houston', 77), ('Oklahoma City', 'Kansas City', 58), ('Salt Lake City', 'Kansas City', 56), ('Atlanta', 'Portland', 53), ('Atlanta', 'Santa Fe', 53), ('Denver', 'Vancouver', 53), ('New York', 'New Orleans', 53), ('Santa Fe', 'Saint Louis', 53), ('Santa Fe', 'Seattle', 53), ('Salt Lake City', 'Oklahoma City', 52)]
[('Boston', 'Houston', 77), ('Oklahoma City', 'Kansas City', 58), ('Salt Lake City', 'Kansas City', 56), ('Atlanta', 'Portland', 53), ('Atlanta', 'Santa Fe', 53), ('Denver', 'Vancouver', 53), ('New York', 'New Orleans', 53), ('Santa Fe', 'Seattle', 53), ('Salt Lake City', 'Oklahoma City', 52), ('San Francisco', 'Santa Fe', 48)]


In [7]:
def capitlize_all_words_after_space(df):
    cols = set(df.columns)
    for column in ('from','to'):
        if column in cols:
            df[column] = df[column].str.title().str.strip()
    return df

In [8]:
def text_pipeline(df):
    df = df.copy()
    capitlize_all_words_after_space(df)
    return df

In [9]:
roads1 = text_pipeline(roads)
dests1 = text_pipeline(dests)
for dataset in(roads1,dests1):
    all_city_names = merge_unique_from_to(dataset)
    most_simelar = show_most_simelar_strings(all_city_names)
    print(most_simelar[:10])

[('Boston', 'Houston', 77), ('Oklahoma City', 'Kansas City', 58), ('Salt Lake City', 'Kansas City', 56), ('Atlanta', 'Portland', 53), ('Atlanta', 'Santa Fe', 53), ('Denver', 'Vancouver', 53), ('New York', 'New Orleans', 53), ('Santa Fe', 'Saint Louis', 53), ('Santa Fe', 'Seattle', 53), ('Salt Lake City', 'Oklahoma City', 52)]
[('Boston', 'Houston', 77), ('Oklahoma City', 'Kansas City', 58), ('Salt Lake City', 'Kansas City', 56), ('Atlanta', 'Portland', 53), ('Atlanta', 'Santa Fe', 53), ('Denver', 'Vancouver', 53), ('New York', 'New Orleans', 53), ('Santa Fe', 'Seattle', 53), ('Salt Lake City', 'Oklahoma City', 52), ('San Francisco', 'Santa Fe', 48)]


In [10]:
def merge_2df(df1,df2):
    return np.concatenate((df1['from'].values,df1['to'].values,df2['from'].values,df2['to'].values))

In [11]:
all_city_names = merge_2df(roads1,dests1)
most_simelar = show_most_simelar_strings(all_city_names)
print(most_simelar[:10])

[('Boston', 'Houston', 77), ('Oklahoma City', 'Kansas City', 58), ('Salt Lake City', 'Kansas City', 56), ('Atlanta', 'Portland', 53), ('Atlanta', 'Santa Fe', 53), ('Denver', 'Vancouver', 53), ('New York', 'New Orleans', 53), ('Santa Fe', 'Saint Louis', 53), ('Santa Fe', 'Seattle', 53), ('Salt Lake City', 'Oklahoma City', 52)]


In [12]:
def fix_spelling_errors(df):
    cols = set(df.columns)
    for column in ('from','to'):
        df[column] = df[column].str.replace('Oklahima','Oklahoma',regex=False)
    return df

def text_pipeline(df):
    df = df.copy()
    df = capitlize_all_words_after_space(df)
    df = fix_spelling_errors(df)
    return df

In [13]:
roads2 = text_pipeline(roads)
dests2 = text_pipeline(dests)
all_city_names = merge_2df(roads2,dests2)
most_simelar = show_most_simelar_strings(all_city_names)
print(most_simelar[:10])

[('Boston', 'Houston', 77), ('Oklahoma City', 'Kansas City', 58), ('Salt Lake City', 'Kansas City', 56), ('Atlanta', 'Portland', 53), ('Atlanta', 'Santa Fe', 53), ('Denver', 'Vancouver', 53), ('New York', 'New Orleans', 53), ('Santa Fe', 'Saint Louis', 53), ('Santa Fe', 'Seattle', 53), ('Salt Lake City', 'Oklahoma City', 52)]


In [14]:
text_pipeline(roads)

,from,to,length,color
0,Seattle,Calgary,4,gray
1,Seattle,Helena,6,yellow
2,Portland,Salt Lake City,6,blue
3,Montreal,Sault St. Marie,5,black
4,Pittsburgh,Chicago,3,orange
...,...,...,...,...
94,Montreal,Boston,2,gray
95,Montreal,Boston,2,gray
96,New York,Boston,2,red
97,New York,Boston,2,yellow


In [15]:
roads2.head()

,from,to,length,color
0,Seattle,Calgary,4,gray
1,Seattle,Helena,6,yellow
2,Portland,Salt Lake City,6,blue
3,Montreal,Sault St. Marie,5,black
4,Pittsburgh,Chicago,3,orange


In [16]:
dests2.head()

,from,to,points
0,Boston,Miami,12
1,Calgary,Phoenix,13
2,Calgary,Salt Lake City,7
3,Chicago,New Orleans,7
4,Chicago,Santa Fe,9


In [17]:
roads2 = roads2[['from','to','length','color']]
dests2 = dests2[['from','to','points']]

In [18]:
roads2.to_csv('Railroads.csv',index=False)
dests2.to_csv('destinations.csv',index=False)